In [1]:
import os

import pymongo

from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support.ui import Select

import pandas as pd

import re
from pprint import pprint
from datetime import datetime
import time

import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey, Date, DateTime, Boolean, BigInteger
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy import inspect

In [3]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.social_blade

In [4]:
collection = mongoDb.social_blade_test

In [5]:
executable_path = {'executable_path': '/Users/soria/Anaconda3/Scripts/chromedriver'}
browser = Browser('chrome', **executable_path)


In [6]:
if mongoDb.social_blade_test.find_one():
    html = mongoDb.social_blade_test.find_one()['social_blade_html']
else:
    url = "https://socialblade.com/youtube/user/GentleWhispering/monthly"
    browser.visit(url)
    html = browser.html
test_html_dict = {'social_blade_url': url, 'social_blade_html':html}
collection.update_one({}, {'$set': test_html_dict}, upsert=True)

blade_soup = bs(html, 'html.parser')


In [7]:
sortdiv = blade_soup.find('div', {'id': 'sort-by'})
sortdiv.next_sibling.next_sibling.next_sibling.next_sibling


AttributeError: 'NoneType' object has no attribute 'next_sibling'

In [ ]:
divs = blade_soup.find_all('div')
for index, div in enumerate(divs):
    print(f"{index}")
    print("========================")
    print(f"{div}")

In [ ]:
the_div = divs[177:178]
the_div

In [ ]:
spans = the_div[0].find_all('span')
for span in spans:
    print(span.text)

In [ ]:
anchors = the_div[0].find_all('a')
for anchor in anchors:
    print(anchor['href'])

In [3]:
url = "https://socialblade.com/youtube/user/GentleWhispering/monthly"
browser.visit(url)
html = browser.html
blade_soup = bs(html, 'html.parser')


In [8]:
script_divs = blade_soup.find_all('script', {'type': 'text/javascript'})
# for index, script in enumerate(script_divs[9:15]):
#     print(index)
#     print('============================')
#     print(script)
script_divs = script_divs[9:15]


In [5]:
raw_text = script_divs[0].text
front_strip = re.sub(r'\Subs\\\\n"(.*)', "", raw_text)
front_strip

'\t\t\n\tg = new Dygraph(\n\n\t// containing div\n\tdocument.getElementById(\'DailySubscribers\'),\n\t// CSV or path to a CSV file.\n\t"Date,Daily Subs\\n" + "2016-02-15,702\\n" + "2016-02-16,455\\n" + "2016-02-17,549\\n" + "2016-02-18,576\\n" + "2016-02-19,563\\n" + "2016-02-20,633\\n" + "2016-02-21,533\\n" + "2016-02-22,400\\n" + "2016-02-23,619\\n" + "2016-02-24,654\\n" + "2016-02-25,592\\n" + "2016-02-26,587\\n" + "2016-02-27,592\\n" + "2016-02-28,666\\n" + "2016-02-29,484\\n" + "2016-03-01,592\\n" + "2016-03-02,583\\n" + "2016-03-03,488\\n" + "2016-03-04,472\\n" + "2016-03-05,528\\n" + "2016-03-06,609\\n" + "2016-03-07,572\\n" + "2016-03-08,701\\n" + "2016-03-09,665\\n" + "2016-03-10,662\\n" + "2016-03-11,695\\n" + "2016-03-12,563\\n" + "2016-03-13,650\\n" + "2016-03-14,646\\n" + "2016-03-15,633\\n" + "2016-03-16,539\\n" + "2016-03-17,650\\n" + "2016-03-18,600\\n" + "2016-03-19,709\\n" + "2016-03-20,503\\n" + "2016-03-21,766\\n" + "2016-03-22,880\\n" + "2016-03-23,594\\n" + "2016-

In [ ]:

raw_text[139:-330]

In [ ]:
raw_text[139:-497]

In [ ]:
test_list = raw_text[139:-397].split('+')
stripped = [test.strip('"\\n') for test in test_list]
strip_test = re.sub(f'\\\\n"', "", test_list[2])
# strip_test = re.sub(f'"', "", strip_test)
# test_date_val = strip_test.split(',')
# test_date_val[1]
test_list

In [12]:
def dataframer(blob, val_type):
    string_list = blob.split('+')
    string_list = string_list[1:]
    stripped = [string.strip('"\\n') for string in string_list]
    stripped = [(re.sub(f'\\\\n"', "", string)) for string in stripped]
    stripped = [(re.sub(f'"', "", string)) for string in stripped]
    dict_list = []
    for strip in stripped:
        date_val = strip.split(',')
        print(date_val)
        date = date_val[0]
        print(date)
        val = date_val[1]
        print(val)
        dict_list.append({'date': date, val_type: val})
    return dict_list

In [13]:
dataframer(raw_text[139:-497], 'daily_subs')


[' 2016-02-15', '702 ']
 2016-02-15
702 
[' 2016-02-16', '455 ']
 2016-02-16
455 
[' 2016-02-17', '549 ']
 2016-02-17
549 
[' 2016-02-18', '576 ']
 2016-02-18
576 
[' 2016-02-19', '563 ']
 2016-02-19
563 
[' 2016-02-20', '633 ']
 2016-02-20
633 
[' 2016-02-21', '533 ']
 2016-02-21
533 
[' 2016-02-22', '400 ']
 2016-02-22
400 
[' 2016-02-23', '619 ']
 2016-02-23
619 
[' 2016-02-24', '654 ']
 2016-02-24
654 
[' 2016-02-25', '592 ']
 2016-02-25
592 
[' 2016-02-26', '587 ']
 2016-02-26
587 
[' 2016-02-27', '592 ']
 2016-02-27
592 
[' 2016-02-28', '666 ']
 2016-02-28
666 
[' 2016-02-29', '484 ']
 2016-02-29
484 
[' 2016-03-01', '592 ']
 2016-03-01
592 
[' 2016-03-02', '583 ']
 2016-03-02
583 
[' 2016-03-03', '488 ']
 2016-03-03
488 
[' 2016-03-04', '472 ']
 2016-03-04
472 
[' 2016-03-05', '528 ']
 2016-03-05
528 
[' 2016-03-06', '609 ']
 2016-03-06
609 
[' 2016-03-07', '572 ']
 2016-03-07
572 
[' 2016-03-08', '701 ']
 2016-03-08
701 
[' 2016-03-09', '665 ']
 2016-03-09
665 
[' 2016-03-10', 

[' 2017-09-15', '715 ']
 2017-09-15
715 
[' 2017-09-16', '783 ']
 2017-09-16
783 
[' 2017-09-17', '737 ']
 2017-09-17
737 
[' 2017-09-18', '596 ']
 2017-09-18
596 
[' 2017-09-19', '772 ']
 2017-09-19
772 
[' 2017-09-20', '821 ']
 2017-09-20
821 
[' 2017-09-21', '665 ']
 2017-09-21
665 
[' 2017-09-22', '559 ']
 2017-09-22
559 
[' 2017-09-23', '704 ']
 2017-09-23
704 
[' 2017-09-24', '567 ']
 2017-09-24
567 
[' 2017-09-25', '645 ']
 2017-09-25
645 
[' 2017-09-26', '662 ']
 2017-09-26
662 
[' 2017-09-27', '669 ']
 2017-09-27
669 
[' 2017-09-28', '542 ']
 2017-09-28
542 
[' 2017-09-29', '553 ']
 2017-09-29
553 
[' 2017-09-30', '609 ']
 2017-09-30
609 
[' 2017-10-01', '639 ']
 2017-10-01
639 
[' 2017-10-02', '602 ']
 2017-10-02
602 
[' 2017-10-03', '592 ']
 2017-10-03
592 
[' 2017-10-04', '588 ']
 2017-10-04
588 
[' 2017-10-05', '491 ']
 2017-10-05
491 
[' 2017-10-06', '696 ']
 2017-10-06
696 
[' 2017-10-07', '510 ']
 2017-10-07
510 
[' 2017-10-08', '650 ']
 2017-10-08
650 
[' 2017-10-09', 

[' 2019-01-28', '1184 ']
 2019-01-28
1184 
[' 2019-01-29', '1156 ']
 2019-01-29
1156 
[' 2019-01-30', '1096 ']
 2019-01-30
1096 
[' 2019-01-31', '991 ']
 2019-01-31
991 
[' 2019-02-01', '1083 ']
 2019-02-01
1083 
[' 2019-02-02', '1063 ']
 2019-02-02
1063 
[' 2019-02-03', '1064 ']
 2019-02-03
1064 
[' 2019-02-04', '624 ']
 2019-02-04
624 
[' 2019-02-05', '785 ']
 2019-02-05
785 
[' 2019-02-06', '820 ']
 2019-02-06
820 
[' 2']
 2


IndexError: list index out of range

In [ ]:
back_strip = re.search(r'^(.*?)Gained', raw_text).group(0)

In [ ]:
test = re.search(r'.+?(?=Gained)', raw_text).group(0)
test

In [ ]:
test2 = re.search(r'(?<=Gained).*', raw_text).group(0)
test2